## Prueba Técnica Datta
### Computer Vision ----> Clasificación de Números realizados con la Mano

 Código utilizado para el entrenamiento de una red neuronal convolucional __CNN__ con __TensorFlow__

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
import numpy as np 
import random
import os
import cv2

In [13]:
# Descarga de datos
dataset_url = "https://trascenderglobaldocs.s3.amazonaws.com/admisiones/64x64_SIGNS.zip"
data_zip = tf.keras.utils.get_file(origin=dataset_url, 
                                          fname='64x64_SIGNS.zip', 
                                          extract=True)
# Directorio donde se almacenan los datos
data_dir = os.path.dirname(data_zip) + '/64x64_SIGNS' 

In [14]:
# Tamaño de las Imágenes
img_size = 64
# Lista para Datos de Entrenamiento
training_data = []
# Lista para Datos de Validación
val_data = []

# Función para convertir Imagen a Arreglo
def img_to_array():
    # Listas para Datos de Entrenamiento
    X = []
    y = []
    # Listas para Datos de Validación
    X_val = []
    y_val = []
    
    # Directorio Datos de Entrenamiento
    path = os.path.join(data_dir, 'train_signs')
    
    for img in os.listdir(path):
            new_path = os.path.join(path, img)
            
            try:
                img_array = cv2.imread(new_path) # Leer imagen
                label = int(img[0]) # Etiqueta de la imagen
                training_data.append([img_array, label]) # Adición como Dato de Entrenamiento
            except Exception as e:
                print(e)
                
    random.shuffle(training_data) # Mezcla de Datos de Entrenamiento
    
    # Separación de datos con etiqueta
    for samples, labels in training_data:
        X.append(samples)
        y.append(labels)
        
    # Convertir lista a np.array    
    X = np.array(X)
    y = np.array(y)
    
    # Guardar Datos para Entrenamiento
    np.save('samples', X)
    np.save('labels', y)
    
    # Se repite el proceso para los datos de Validación
    path = os.path.join(data_dir, 'val_signs')
    
    for img in os.listdir(path):
            new_path = os.path.join(path, img)
            
            try:
                img_array = cv2.imread(new_path)
                label = int(img[0])
                val_data.append([img_array, label])
            except Exception as e:
                print(e)
                
    random.shuffle(val_data)
    
    for samples, labels in val_data:
        X_val.append(samples)
        y_val.append(labels)
        
    X_val = np.array(X_val)
    y_val = np.array(y_val)
    
    print(y.shape)
    print(y_val.shape)
    np.save('samples_val', X_val)
    np.save('labels_val', y_val)

In [15]:
# Función para definir vector de Salida de Categorías
def to_categorical(array):
    one_hot = np.zeros((len(array), 6)) # 6 es el número de categorías
    
    for idx, num in enumerate(array):
        one_hot[idx][num] = 1 # Se asigna 1 a la posición del array que corresponde a su etiqueta (label)
    return one_hot

In [16]:
def build_model():
    model = Sequential()

    model.add(Conv2D(64, (3,3), input_shape=(64,64,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(128, (3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(128, (3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
        
    model.add(Flatten())

    model.add(Dense(6))
    model.add(Activation('softmax'))

    return model

In [17]:
# Función para Entrenamiento del modelo
def train():
    # Cargar Datos de Entrenamiento
    samples = np.load('samples.npy')
    labels = to_categorical(np.load('labels.npy'))
    
    # Cargar Datos de Validación
    samples_val = np.load('samples_val.npy')
    labels_val = to_categorical(np.load('labels_val.npy'))
    
    # Normalización de los Datos (0-1)
    samples = samples.astype('float32') / 255
    samples_val = samples_val.astype('float32') / 255
    
    # Entrenamiento
    network = build_model()
    
    network.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['accuracy'])
    
    network.fit(samples, labels, batch_size=64, epochs=20, validation_data=(samples_val, labels_val))

    network.save('Network.model')
    

In [18]:
# Si ya se tienen los Datos de Entrenamiento, se procede al Entrenamiento
if (os.path.isfile('samples.npy') and os.path.isfile('labels.npy')):
    train()
    #Sino, se obtienen los datos y luego se entrena
else:
    img_to_array()
    train()

Epoch 1/20
27/27 [==============================] - 29s 1s/step - loss: 1.8448 - accuracy: 0.2059 - val_loss: 1.6631 - val_accuracy: 0.4167
Epoch 2/20
27/27 [==============================] - 22s 830ms/step - loss: 1.5040 - accuracy: 0.4238 - val_loss: 1.4534 - val_accuracy: 0.3935
Epoch 3/20
27/27 [==============================] - 22s 833ms/step - loss: 1.1480 - accuracy: 0.5749 - val_loss: 1.0812 - val_accuracy: 0.5648
Epoch 4/20
27/27 [==============================] - 28s 1s/step - loss: 0.9139 - accuracy: 0.6604 - val_loss: 0.7835 - val_accuracy: 0.7222
Epoch 5/20
27/27 [==============================] - 23s 842ms/step - loss: 0.6701 - accuracy: 0.7679 - val_loss: 0.6412 - val_accuracy: 0.7731
Epoch 6/20
27/27 [==============================] - 22s 832ms/step - loss: 0.4733 - accuracy: 0.8439 - val_loss: 0.6418 - val_accuracy: 0.7778
Epoch 7/20
27/27 [==============================] - 29s 1s/step - loss: 0.4018 - accuracy: 0.8867 - val_loss: 0.6316 - val_accuracy: 0.7917
Epoch 8/